In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
import networkx as nx
import time
import itertools

In [ ]:
deputados = pd.read_csv('deputados.csv', delimiter=';')
deputados

In [ ]:
deputados.columns

In [ ]:
legislaturas = pd.read_csv('legislaturas.csv', delimiter=';')
legislaturas

In [ ]:
budget_2019 = pd.read_csv('Ano-2019.csv', delimiter=';')
budget_2020 = pd.read_csv('Ano-2020.csv', delimiter=';')
budget_2021 = pd.read_csv('Ano-2021.csv', delimiter=';')
budget_2022 = pd.read_csv('Ano-2022.csv', delimiter=';')
budget = pd.concat([budget_2019.copy(), budget_2020.copy(), budget_2021.copy(), budget_2022.copy()])

In [ ]:
budget.columns

In [ ]:
parties = budget.groupby('txNomeParlamentar')['sgPartido'].unique().reset_index()
parties

In [ ]:
parties[['Partido1', 'Partido2']] = parties['sgPartido'].astype(str).str.strip("[]").astype(str).str.split(pat=" ", expand=True)
parties

In [ ]:
# Remove "LIDERANÇAS", which have party set to 'nan'
parties = parties[parties['Partido1'] != 'nan']

In [ ]:
parties_pivot = parties.pivot(index='txNomeParlamentar', columns=['Partido1'], values=['Partido1', 'Partido2'])
parties_pivot

In [ ]:
list(zip(parties['txNomeParlamentar'], parties['Partido1'].str.strip("'")))

In [ ]:
parties_pile = parties.melt(id_vars='txNomeParlamentar', value_vars=['Partido1', 'Partido2'], var_name='PartidoNum', value_name='PartidoSigla')
parties_pile_valid = parties_pile[~parties_pile['PartidoSigla'].isna()].copy()
parties_pile_valid

In [ ]:
parties_pile_valid['Edges'] = list(zip(parties_pile_valid['txNomeParlamentar'], parties_pile_valid['PartidoSigla'].str.replace("'", "")))
parties_pile_valid

In [ ]:
# https://networkx.org/nx-guides/content/exploratory_notebooks/facebook_notebook.html#basic-topological-attributes
# https://networkx.org/documentation/stable/reference/index.html

graph = nx.Graph()
for _name in parties['txNomeParlamentar']:
    graph.add_node(_name)
for _edge in parties_pile_valid['Edges']:
    graph.add_edge(*_edge) # *_edge to unpack the tuple

In [ ]:
# Creating the parties cmap according to political spectrum
parties_cmap = {'left': "#D8113A", 'center-left': "#F25B92", 'center': "#FFBC42", 'center-right': "#0496FF", 'right': "#006BA6"}
representative_cmap = "#6c757d"

# Assigning spectrum to parties
parties_spectrum = {'DEM': 'center-right',
                    'PODE': 'center-right',
                    'AVANTE': 'center',
                    'PT': 'center-left',
                    'PR': 'right',
                    'PRB': 'right',
                    'PP': 'right',
                    'PTB': 'right',
                    'PSC': 'right',
                    'PROS': 'center',
                    'PDT': 'center-left',
                    'PV': 'center-left',
                    'PSDB': 'center-right',
                    'PPS': 'center-left',
                    'PSD': 'center',
                    'SOLIDARIEDADE': 'center',
                    'UNIÃO': 'center-right',
                    'MDB': 'center-right',
                    'NOVO': 'right',
                    'PATRIOTA': 'right',
                    'PSB': 'center-left',
                    'CIDADANIA': 'center-left',
                    'REPUBLICANOS': 'right',
                    'PCdoB': 'left',
                    'PL': 'right',
                    'PATRI': 'right',
                    'PHS': 'center-right',
                    'PSL': 'right',
                    'PSOL': 'left',
                    'REDE': 'center-left',
                    'S.PART.': 'center',
                    'PPL': 'left'}

# Using the dict keys to finally create a list of acronyms
parties_acronyms = list(parties_spectrum.keys())

def assign_color(node: str) -> str:
    """Return an entry for the node color list, based on party spectrum or representative.
    """
    color: str = ""
    if node in parties_acronyms:
        color = parties_cmap[parties_spectrum[node]]
    else:
        color = representative_cmap
        
    return color

# Creating the list of colors for each node
node_colors = [assign_color(node) for node in graph.nodes()]

In [ ]:
# Creating different sizes for nodes that are parties and nodes that are representatives
node_sizes = [300 if node in parties_acronyms else 10 for node in graph.nodes()]

In [ ]:
pos = nx.spring_layout(graph, seed=61189, iterations=25)

In [ ]:
plot_options = {"with_labels": False, "width": 0.15, "alpha":0.6}
fig, ax = plt.subplots(figsize=(16, 9))
ax.axis("off")
nx.draw_networkx(G=graph, pos=pos, ax=ax, node_size=node_sizes, **plot_options)

In [ ]:
plot_options = {"with_labels": False, "width": 0.15, "alpha":1.0}
fig, ax = plt.subplots(figsize=(16, 9))
ax.axis("off")
nx.draw_networkx(G=graph, pos=pos, ax=ax, node_color=node_colors, node_size=node_sizes, **plot_options)

In [ ]:
# "Self referencing" dict, only for the parties names to appear as labels in the graph
parties_labels_pos = dict(zip(list(pos.keys())[-32:], list(pos.keys())[-32:]))

In [ ]:
plot_options = {"with_labels": False, "width": 0.15, 'alpha': 0.7}
fig, ax = plt.subplots(figsize=(16, 9))
ax.axis("off")
nx.draw_networkx(G=graph, pos=pos, ax=ax, node_color=node_colors, node_size=node_sizes, **plot_options)
nx.draw_networkx_labels(graph, pos, parties_labels_pos, font_size=10, font_weight='bold', horizontalalignment='left', verticalalignment='top')

# LEGEND (customized)
legend_elements = []
for party in parties_cmap.keys():
    legend_elements.append(
        Line2D(
            [], [],
            marker='o',
            markersize=12,
            markeredgecolor=parties_cmap[party],
            markerfacecolor=parties_cmap[party],
            linestyle='',
            label=str.capitalize(party)
        )
    )
fig.legend(handles=legend_elements, frameon=False, loc='lower center', ncol=len(parties_cmap), fontsize=13, handletextpad=0.1)

In [ ]:
party_centrality = pd.DataFrame.from_dict(nx.centrality.degree_centrality(graph), orient='index', columns=['Centrality'])
party_centrality['Spectrum'] = party_centrality.index.map(parties_spectrum).astype(str).map(str.capitalize)
# party_centrality[party_centrality.index.isin(parties_acronyms)].sort_values(by='Centrality', ascending=False).apply(lambda t: round(t * 100, 2))
party_centrality['Centrality %'] = (
    party_centrality[party_centrality.index.isin(parties_acronyms)]['Centrality']
    .apply(lambda t: round(t * 100, 2))
)

party_centrality[party_centrality.index.isin(parties_acronyms)].sort_values(by='Centrality', ascending=False)[['Spectrum', 'Centrality %']].head(10)


In [ ]:
pd.DataFrame.from_dict(nx.centrality.betweenness_centrality(graph), orient='index', columns=['Betweeness']).sort_values(by='Betweeness', ascending=False).head(30)

In [ ]:
pd.DataFrame.from_dict(nx.centrality.closeness_centrality(graph), orient='index', columns=['Closeness']).sort_values(by='Closeness', ascending=False).head(30)